
Veri işleme ve analizi için pandas kütüphanesi kullanılacaktır. Pandas kütüphanesi pd rumuzu ile kısaltılmıştır.


In [1]:
import pandas as pd

Verilerimiz Google Drive (bulut) üzerinde olduğu için Google Colab ile Google Drive arasında bağlantı kurulacaktır.

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Google Drive içinde Colab Notebooks klasörünün içinde TDDI2022 isimli klasörde verilerimizi sakladığımız için bu klasöre erişilecektir.

NOT: Bu klasörü daha önce oluşturmadıysanız %md TDDI2022 komutu veya Google Drive Ara Yüzü ile klasörü oluşturunuz!

In [ ]:
%cd drive/
%cd MyDrive/
%cd Colab Notebooks/
%cd TDDI2022/

%ls

Veri İskeleti TDDI2022 klasöründe bulunan yarışma kapsamında yarışmacılara sağlanan kanunum-nlp-doc-analysis-dataset.csv dosyasının içindeki verileri pandas ile okuyarak oluşturulacaktır.

Veri iskeleti oluşturulduktan sonra veri iskeleti ile ilgili bilgi ekrana yazdırılacaktır. Tam özetin yazdırılıp yazdırılmayacağı durumunu verbose=True (yazdırılsın) parametresi ile kontrol edilecektir.

In [ ]:
df = pd.read_csv('kanunum-nlp-doc-analysis-dataset.csv')
df.info(verbose=True)

In [ ]:
for i in range(4142) : 
  if df.loc[i]["kategori"] == "Özelge":
    print(i)

TDDI için transformers kütüphanesinin 4.20.1 versiyonu indirilmektedir.

In [ ]:
!pip install transformers==4.20.1

In [ ]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, pipeline

modeller = ["yunusemreemik/logo-qna-model","savasy/bert-base-turkish-squad","lserinol/bert-turkish-question-answering"]

icerik = df['data_text'][0][:210]
sorular = ["Resmi Gazete Tarihi nedir?","Resmî Gazete Sayısı nedir?", "Kanunun No nedir?","Kabul Tarihi nedir?","Kanunun başlığı nedir?"]


for X in modeller:
  print("Model:",X)
  veriBulucu = pipeline(task='question-answering', model=AutoModelForQuestionAnswering.from_pretrained(X), tokenizer=AutoTokenizer.from_pretrained(X))
  for soru in sorular:
    sonuc = veriBulucu(question = soru, context = icerik)
    print("'",soru,"'","sorusuna verilen cevap:",sonuc['answer'])

Tarih verilerinin formatlarının farklı olması nedeniyle hapsini aynı şekildeki bir stringe dönüştürüp karşılaştırma yapılmasını sağlamaktadır.

In [8]:
#tarih = input("tarih gir")

def tarihduzelt(tarih):
  ayracsayisi = 0 
  i = 0
  gun = ''
  ay = ''
  yil = ''
  if tarih.find('-') != -1:
      ayracsayisi = 2
  while i < len(tarih):
    if tarih[i].isdigit() and ayracsayisi == 0: 
      gun = gun + tarih[i]
    if tarih[i].isdigit() and ayracsayisi == 1:
      ay = ay + tarih[i]
    if tarih[i].isdigit() and ayracsayisi == 2:
      yil = yil + tarih[i]

    if tarih[i] == '/' or tarih[i] == '.':
      ayracsayisi += 1
    if tarih[i] == '-':
      ayracsayisi -= 1
    i += 1

  if len(gun) < 2:
    gun = '0' + gun
  if len(ay) < 2:
    ay = '0' + ay
  return gun+ay+yil
#print(tarihduzelt(tarih))

In [9]:

def veriduzelt(girdi):
  if len(girdi) > 10:
    girdi = girdi[len(girdi)-11:len(girdi)]
    i = len(girdi) - 1
  else:
    return girdi
  gecici = ''
  while True and i:
    if girdi[i].isdigit():
      gecici += girdi[i]
    i -= 1
    if i == 0:
      break
  sonuc = ''
  for j in range(len(gecici)):
    sonuc += gecici[len(gecici)-(j+1)]
  return sonuc
  

In [ ]:
print(df["data_text"][1939][:310])
print(df["rega_no"][1939][:310])

In [ ]:
modeller = ["savasy/bert-base-turkish-squad"]
kr_trh_dogru=0
kr_trh_yanlis=0

for i in range (3158,4142):
  icerik = df['data_text'][i][:350]
  sorular = ["Belge Tarihi nedir?"]
  if df["kategori"][i] == "Özelge":
    for X in modeller:
      print(i)
      veriBulucu = pipeline(task='question-answering', model=AutoModelForQuestionAnswering.from_pretrained(X), tokenizer=AutoTokenizer.from_pretrained(X))
      for soru in sorular:
        sonuc = veriBulucu(question = soru, context = icerik)
      
        if soru == "Belge Tarihi nedir?" :
          kr_trh1 = sonuc['answer']                                                                                                                        
        print("'",soru,"'","sorusuna verilen cevap:",sonuc['answer']) 

    kr_trh2 = df["mevzuat_tarihi"][i]
    kr_trh1 = tarihduzelt(kr_trh1)
    kr_trh2 = tarihduzelt(kr_trh2)


  
    if kr_trh1 == kr_trh2:
      kr_trh_dogru += 1
    else:
      kr_trh_yanlis += 1
      print("*kr_trh*  cevap: ", kr_trh1, "doğrusu: ", kr_trh2)
  

    print(" kr_trh_dogru/yanlis:", kr_trh_dogru, kr_trh_yanlis)
    print("---------------------------------------------------------")
